In [54]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import pyodbc
database = 'hackathon_danone'
username = 'hack12'
password = 'boA8AKUb54!'
driver= '{SQL SERVER}'
server = '192.168.250.3'

my_engine = create_engine(f"""mssql+pyodbc://{username}:{password}@{server}:1433/{database}?driver={server}', fast_executemany=True""" )

def db_sql(sql):
    cnxn = pyodbc.connect(f'DRIVER={driver};PORT=1433;SERVER={server};DATABASE={database};UID={username};PWD={password}')
    data = pd.read_sql(sql,cnxn)
    cnxn.close()    
    return data

In [29]:
raw_mat_in = db_sql("SELECT * from dbo.recipe_0_raw_material_in")

raw_mat_used = db_sql("SELECT * from dbo.recipe_0_raw_material_used")

raw_materials = raw_mat_in.merge(raw_mat_used, on=['id', 'process_order_sap3'])

In [30]:
cols = ['fat_pct', 'particles_grp1', 'particles_grp2',
                              'particles_grp3', 'moisture']

In [31]:
for col in cols:
    raw_materials[col] *= raw_materials['usage_pct']

In [36]:
raw_materials = raw_materials.groupby('id').apply(sum)

In [ ]:
out = db_sql("SELECT * from dbo.recipe_0_out_test_during_production")

In [9]:
out['bulk_density'] = pd.to_numeric(out['bulk_density'])

In [11]:
out_gr = out.groupby('orders_details_id').agg("mean")[['moisture', 'bulk_density']]

In [38]:
df = out_gr.join(raw_materials, lsuffix='_out')

In [40]:
from sklearn.linear_model import LinearRegression

In [41]:
lr = LinearRegression()

In [61]:
cols = ['fat_pct', 'particles_grp1', 'particles_grp2', 'particles_grp3', 'moisture']

In [62]:
lr.fit(df[cols], df['bulk_density'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [66]:
np.sqrt((abs(df['bulk_density'] - lr.predict(df[cols])) ** 2).mean())

7.31019212374755